<a href="https://colab.research.google.com/github/AnanyarajS/Machine_Learning_Projects/blob/main/Logistic_Regression_Model/logestic_regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import numpy as np
import pandas as pd

In [70]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [71]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [72]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [73]:
train = train.drop(columns='Name')
test = test.drop(columns='Name')

In [74]:
cols = [
    'Age',
    'Fare'
]
train[cols] = train[cols].fillna(train[cols].median())
test[cols] = test[cols].fillna(train[cols].median())

In [75]:
train['Deck'] = train['Cabin'].str[0]
test['Deck'] = train['Cabin'].str[0]


train['Deck'] = train['Deck'].fillna('U')
test['Deck'] = test['Deck'].fillna('U')

In [76]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

In [77]:
train = train.drop(columns='Cabin')
test = test.drop(columns='Cabin')

In [78]:
train['Sex'] = train['Sex'].map({'male':0, 'female':1})
test['Sex'] = test['Sex'].map({'male':0, 'female':1})

In [79]:
train = train.drop(columns='Ticket')
test = test.drop(columns='Ticket')

In [80]:
train = pd.get_dummies(train, columns=['Embarked', 'Deck', 'Pclass'], drop_first=True)
test = pd.get_dummies(test, columns=['Embarked', 'Deck', 'Pclass'], drop_first=True)

In [81]:
dummy_cols = [col for col in train.columns if col.startswith('Embarked_') or col.startswith('Deck_') or col.startswith('Pclass_')]
train[dummy_cols] = train[dummy_cols].astype(int)
test[dummy_cols] = test[dummy_cols].astype(int)


In [82]:
train['Age'] = (train['Age']-train['Age'].mean())/train['Age'].std()
test['Age'] = (test['Age']-train['Age'].mean())/train['Age'].std()
train['Fare'] = (train['Fare']-train['Fare'].mean())/train['Fare'].std()
test['Fare'] = (test['Fare']-train['Fare'].mean())/train['Fare'].std()

In [83]:
train.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_U,Pclass_2,Pclass_3
0,1,0,0,-0.565419,1,0,-0.502163,0,1,0,0,0,0,0,0,0,1,0,1
1,2,1,1,0.663488,1,0,0.786404,0,0,0,1,0,0,0,0,0,0,0,0
2,3,1,1,-0.258192,0,0,-0.488580,0,1,0,0,0,0,0,0,0,1,0,1
3,4,1,1,0.433068,1,0,0.420494,0,1,0,1,0,0,0,0,0,0,0,0
4,5,0,0,0.433068,0,0,-0.486064,0,1,0,0,0,0,0,0,0,1,0,1


In [84]:
test.head()

,PassengerId,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_U,Pclass_2,Pclass_3
0,892,0,34.5,0,0,7.8292,1,0,0,0,0,0,0,0,0,1,0,1
1,893,1,47.0,1,0,7.0000,0,1,0,1,0,0,0,0,0,0,0,1
2,894,0,62.0,0,0,9.6875,1,0,0,0,0,0,0,0,0,1,1,0
3,895,0,27.0,0,0,8.6625,0,1,0,1,0,0,0,0,0,0,0,1
4,896,1,22.0,1,1,12.2875,0,1,0,0,0,0,0,0,0,1,0,1
